In [256]:
import pandas as pd
import numpy as np

from google.colab import drive

# Bibliotecas de pré-processamento
from sklearn.preprocessing import LabelEncoder

# Bibliotecas de plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [257]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [258]:
df = pd.read_csv('/content/drive/MyDrive/Data/merged_df.csv')

In [259]:
df = df.drop(['record_id', 'idade_faixa'], axis=1)

In [260]:
X_afetam_eficacia = df[['idade_diagnostico','primeiro_IMC','ultima_informacao_paciente', 'tempo_seguimento', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local']]

In [261]:
from sklearn.preprocessing import StandardScaler
X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])

In [262]:
from sklearn.cluster import KMeans

km = KMeans( n_clusters = 2, init = 'random', max_iter = 300, n_init = 100, random_state = 52 )

In [263]:
km.fit( X_afetam_eficacia )

KMeans(init='random', n_clusters=2, n_init=100, random_state=52)

In [264]:
y_km = km.predict( X_afetam_eficacia )
df['eficacia_tratamento'] = y_km

In [265]:
df[['idade_diagnostico','ultima_informacao_paciente', 'tempo_seguimento', 'grau_histologico', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local', 'primeiro_peso', 'eficacia_tratamento']]

,idade_diagnostico,ultima_informacao_paciente,tempo_seguimento,grau_histologico,recidiva_distancia,recidiva_regional,recidiva_local,primeiro_peso,eficacia_tratamento
0,51.00,0.00,2225.00,2.00,0.00,0.00,0.00,57.80,0
1,56.00,3.00,4153.00,3.00,0.00,0.00,0.00,84.00,1
2,42.00,3.00,3290.00,2.00,0.00,0.00,0.00,63.20,0
3,52.00,2.00,182.00,3.00,0.00,0.00,0.00,54.00,0
4,40.00,3.00,3234.00,2.00,1.00,0.00,0.00,58.00,0
...,...,...,...,...,...,...,...,...,...
3171,69.00,3.00,875.00,2.00,0.00,0.00,0.00,75.55,1
3172,60.00,3.00,859.00,3.00,0.00,0.00,0.00,73.00,1
3173,47.00,3.00,873.00,1.00,0.00,0.00,0.00,113.85,0
3174,50.00,3.00,869.00,2.00,0.00,0.00,0.00,57.00,0


In [266]:
# fazer alguns graficos de que tinha ((idade avançada com primeiro peso grande e eficacia tratamento))

In [267]:
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 1")))
# 0 é tratamento bom 
# 1 é tratamento ruim
# isso mostra dois casos o de uma pessoa mais idosa de 80 anos de idade que saiu viva ou seja o tratamento foi bom 
#o outro mostra que não tem nenhum caso em que a pessoa mais idosa sobreviveu e o tratamento foi classificado como negativo

0
38


In [268]:
df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")

print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")))
# 0 é tratamento bom 
# 1 é tratamento ruim
# isso mostra dois casos o de uma mulher nova com menos de 30 anos uma que morreu e a outra que não de maneira que ela morreu, então o tratamento
# bem classificado, ou seja ele esta classificando bem

12
0


In [269]:
df = df.query("tratamento == 0 or tratamento == 1")[:]

In [270]:
df

,escolaridade,idade_diagnostico,sexo,raca,ultima_informacao_paciente,tempo_seguimento,ja_gravida,gestacao_idade,tempo_amamentacao,grau_parentesco_mama_primeiro_1_vez,...,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,diferenca_peso,coef_peso,diferenca_tempo,coef_peso_tempo,eficacia_tratamento
0,1.00,51.00,0.00,3.00,0.00,2225.00,1.00,21.00,23.00,0.00,...,158.00,23.15,64.90,158.00,26.00,7.10,1.12,1602.00,0.00,0
1,1.00,56.00,0.00,2.00,3.00,4153.00,1.00,29.00,39.00,0.00,...,152.00,36.36,85.90,152.00,37.18,1.90,1.02,1455.00,0.00,1
2,2.00,42.00,0.00,4.00,3.00,3290.00,1.00,23.00,8.00,0.00,...,167.00,22.66,63.65,167.00,22.82,0.45,1.01,1616.00,0.00,0
3,5.00,52.00,0.00,0.00,2.00,182.00,1.00,24.00,2.00,0.00,...,146.00,25.33,60.20,147.00,27.86,6.20,1.11,4626.00,0.00,0
4,5.00,40.00,0.00,1.00,3.00,3234.00,1.00,28.00,57.00,0.00,...,148.00,26.48,55.00,150.00,24.44,-3.00,0.95,1341.00,-0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3171,3.00,69.00,0.00,1.00,3.00,875.00,1.00,32.00,32.00,0.00,...,175.00,24.67,62.74,175.00,20.49,-8.15,1.16,752.00,0.00,1
3172,5.00,60.00,0.00,1.00,3.00,859.00,1.00,22.00,20.00,0.00,...,153.00,31.18,82.35,153.00,35.18,9.35,1.13,2685.00,0.00,1
3173,0.00,47.00,0.00,0.00,3.00,873.00,1.00,25.00,18.00,0.00,...,155.00,47.39,43.98,152.65,18.88,5.32,0.92,0.00,0.00,0
3174,3.00,50.00,0.00,1.00,3.00,869.00,1.00,22.00,3.00,0.00,...,158.00,22.83,57.00,158.00,22.83,0.00,1.00,0.00,0.00,0


In [271]:
import numpy as np

df = df.astype('float64')
df = df[np.isfinite(df).all(1)]

In [272]:
from sklearn.model_selection import train_test_split

X = df.drop('tratamento', axis=1)
y = df['tratamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 73)

In [273]:
import lightgbm as lgb

model = lgb.LGBMClassifier()

model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9958448753462604
0.6523143164693218


In [ ]:
!pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

In [281]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.67,0.67,0.67,0.67,1.06
RandomForestClassifier,0.67,0.67,0.67,0.67,2.88
ExtraTreesClassifier,0.66,0.66,0.66,0.66,0.83
RidgeClassifierCV,0.66,0.66,0.66,0.66,0.20
LinearSVC,0.66,0.66,0.66,0.66,0.89
LogisticRegression,0.66,0.66,0.66,0.66,0.16
SVC,0.66,0.66,0.66,0.66,1.79
LGBMClassifier,0.66,0.65,0.65,0.66,1.90
LinearDiscriminantAnalysis,0.66,0.65,0.65,0.66,0.13


In [ ]:
from sklearn.metrics import precision_score

precisao = precision_score(y_true, y_pred)
print(f'Precisão: {precisao:.2f}')

In [278]:
#precisamos fazer com que no treinamento dos bons ele não aceite só numericos mais varios e transforme os não numericos
#normalizar os dados de com standardScaler e minMaxScaler
#definir modelo final com melhores metricas e treinos 